<a href="https://colab.research.google.com/github/Smfatima/Machine-Learning/blob/main/Supervised%20Learning/Classification_using_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classification using Tensorflow of Iris Dataset



In [ ]:
%tensorflow_version 2.x

In [ ]:
#Import libraries
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow.keras

In [ ]:
#Load dataset
csv_coloumn_names = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
Species = ['Setosa', 'Versicolor', 'Viriginica']

train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

train = pd.read_csv(train_path, names=csv_coloumn_names, header = 0)
test = pd.read_csv(test_path, names=csv_coloumn_names, header = 0)


In [ ]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [ ]:
#Input function
def train_input_fn(features, labels, training=True, batch_size=256):
  #convert the input to a dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  #shuffle and repeat if ypu are in training mode
  if training:
    dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

In [ ]:
#feature column
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [ ]:
#Build a DNN with 2 hidden layers

classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    hidden_units = [30, 10], 
    n_classes = 3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmppj3gg2c3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
#Train the model
classifier.train(
    input_fn = lambda: train_input_fn(train, train_y, training = True),
    steps = 5000
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmppj3gg2c3/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.0587089, step = 0
INFO:tensorflow:global_step/sec: 428.801
INFO:tensorflow:loss = 0.9031389, step = 100 (0.235 sec)
INFO:tensorflow:global_step/sec: 520.4
INFO:tensorflow:loss = 0.8555454, step = 200 (0.193 sec)
INFO:tensorflow:global_step/sec: 499.239
INFO:tensorflow:loss = 0.8218584, step = 300 (0.200 sec)
INFO:tensorflow:global_step/sec: 525.191
INFO:tensorflow:loss = 0.79668033, step = 400 (0.190 sec)
INFO:tensorflow:global_step/sec: 555.664
INFO:tensorflow:loss = 0.7681616, step = 500 (0.179 sec)
INFO:te

In [ ]:
 #prediction
 def input_fn(features, batch_size=256):
   #convert the inputs to dataset without labels
   return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']

predict = {}

print("Please type numeric values as prompted.")

for feature in features:
  valid=True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid=False
  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda: input_fn(predict))
for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(
      Species[class_id], 100 * probability))

Please type numeric values as prompted.
SepalLength: 0.5
SepalWidth: 0.6
PetalLength: 0.7
PetalWidth: 0.8
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmppj3gg2c3/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (39.8%)
